In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

data = pd.read_csv('Churn_Modelling.csv')

data = data.head()

In [37]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
label_encoder_gender = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0


In [34]:
from sklearn.preprocessing import OneHotEncoder

oneHotEncode =  OneHotEncoder()
oneHot_data = oneHotEncode.fit_transform(data[['Geography']])

oneHot_data.toarray()


array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [ ]:
oneHotVal = oneHot_data.toarray()
geoName = oneHotEncode.get_feature_names_out(['Geography'])
geoName

array(['Geography_France', 'Geography_Spain'], dtype=object)

In [33]:
oneHotVal = oneHot_data.toarray()
geoName = oneHotEncode.get_feature_names_out(['Geography'])
dataList = pd.DataFrame(oneHotVal, columns=geoName)
dataList

,Geography_France,Geography_Spain
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [38]:
data = pd.concat([data.drop('Geography',axis=1), dataList], axis=1)

data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,1.0,0.0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,1.0,0.0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0


In [44]:

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)


with open('oneHotEncode.pkl', 'wb') as file:
    pickle.dump(oneHotEncode, file)

In [86]:
x = data.drop('Exited', axis=1)
y = data['Exited']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

scalar = StandardScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)

x_train

array([[ 1.44232726,  0.        ,  1.        , -0.45083482,  0.74981839,
        -0.90453403,  0.57735027,  1.        , -1.42390702, -1.73205081,
         1.73205081],
       [-1.30797349,  0.        ,  0.33333333,  1.71317231,  1.22210292,
         1.50755672,  0.57735027, -1.        ,  1.33830627,  0.57735027,
        -0.57735027],
       [-0.38330341,  0.        ,  0.33333333, -0.45083482, -0.98596066,
        -0.90453403,  0.57735027,  1.        ,  0.34092903,  0.57735027,
        -0.57735027],
       [ 0.24894964,  0.        , -1.66666667, -0.81150267, -0.98596066,
         0.30151134, -1.73205081, -1.        , -0.25532829,  0.57735027,
        -0.57735027]])

In [51]:
with open('scalar.pkl', 'wb') as file:
    pickle.dump(scalar, file)

In [88]:
x_train.shape[1]

11

In [87]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                768       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [89]:
import tensorflow
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

opt = Adam(learning_rate=0.01)  # optional

loss = binary_crossentropy  # optional


model.compile(optimizer='Adam', loss="binary_crossentropy",metrics=['accuracy'])

In [96]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_callBack = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [80]:
stopProcess = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [91]:
### Train the model
history = model.fit(
    x_train,y_train,validation_data = (x_test, y_test), epochs = 100,
    callbacks=[log_callBack, stopProcess]
)

Epoch 1/100
1/1 [==============================] - 1s 783ms/step - loss: 0.6233 - accuracy: 0.7500 - val_loss: 0.5677 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 67ms/step - loss: 0.5961 - accuracy: 0.7500 - val_loss: 0.5611 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 87ms/step - loss: 0.5698 - accuracy: 1.0000 - val_loss: 0.5549 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 79ms/step - loss: 0.5448 - accuracy: 1.0000 - val_loss: 0.5486 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 85ms/step - loss: 0.5207 - accuracy: 1.0000 - val_loss: 0.5421 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 80ms/step - loss: 0.4983 - accuracy: 1.0000 - val_loss: 0.5344 - val_accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 83ms/step - loss: 0.4778 - accuracy: 1.0000 - val_loss: 0.5247 - val_accuracy: 1.0000
Epoch 8/100
1/1 [==

In [92]:
model.save('model.h5')

c:\Users\peternrk\Videos\ANN_Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [93]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 4376), started 0:00:28 ago. (Use '!kill 4376' to kill it.)

In [110]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

model = load_model('model.h5')

with open('oneHotEncode.pkl', 'rb') as file:
    oneHotEncode = pickle.load(file)


with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)


with open('scalar.pkl', 'rb') as file:
     scalar = pickle.load(file)

In [124]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [125]:
geo_encoder = oneHotEncode.transform([[input_data['Geography']]]).toarray()
geo_encode_df = pd.DataFrame(geo_encoder, columns=oneHotEncode.get_feature_names_out(['Geography']))
geo_encode_df

c:\Users\peternrk\Videos\ANN_Project\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Spain
0,1.0,0.0


In [128]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Female,40,3,60000,2,1,1,50000


In [ ]:
# label_encoder_gender.fit(input_df['Gender'])
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000
